In [26]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, Normalizer
from keras.losses import CategoricalCrossentropy
from keras.callbacks import EarlyStopping
import tensorflow
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2

In [47]:
validation_df = pd.read_csv("cosas_finales/tiempos_test.csv")
validation_df

,name,N,seconds,N.1,seconds.1,N.2,seconds.2,tipo,tipoProblema
0,-114,100000,0.170155,300000,0.447407,500000,0.713649,0,0
1,-cajasBolas,10000000,0.000480,500000000,0.001063,990000000,0.001211,1,1
2,-167fractales,100000000,0.000558,500000000,0.000962,990000000,0.001198,1,1
3,--OJ10019,2,0.002060,300,0.009723,999,0.023063,1,1
4,--323,90000,0.000480,490000,0.000511,980000,0.001271,1,1
5,niniosPrimero,10,0.001838,100000,0.033445,200000,0.065974,2,2
6,palmerasNieve,10000,0.004839,50000,0.010540,100000,0.016269,2,2
7,-palmerasNieve2,10000,0.007839,50000,0.013541,100000,0.019269,2,2
8,--erasmus,1000,0.006315,50000,0.013816,99000,0.030289,2,2
9,mergeSort,21474,0.018019,107374,0.047188,212600,0.095220,3,3


In [18]:
df = pd.read_csv("cosas_finales/casos_semireal_new_all.txt")
df

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,2584,0.0107,11629,0.0009,24551,0.0008,0
1,1055,0.0031,4750,0.0015,10028,0.0012,0
2,2989,0.0015,13450,0.0011,28395,0.0011,0
3,197,0.0025,886,0.0011,1871,0.0010,0
4,754,0.0013,3393,0.0010,7164,0.0009,0
...,...,...,...,...,...,...,...
11993,97,556.7960,438,50874.6000,926,483428.0000,5
11994,99,595.7910,447,55007.4000,945,401368.0000,5
11995,97,387.8500,438,34860.9000,926,311789.0000,5
11996,98,365.8730,442,34268.2000,933,309820.0000,5


In [79]:
x_validation_data = validation_df.iloc[:, 1:7]
y_validation_data = to_categorical(validation_df['tipo'])

x_validation_data, y_validation_data = np.array(x_validation_data), np.array(y_validation_data)
x_validation_data = np.asarray(x_validation_data).astype('float32')
y_validation_data = np.asarray(y_validation_data).astype('float32')

y_validation_data

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [19]:
#normalize data (using normalizer) scaling cant be used because then individual samples used at the "production" stage cant be normalized
ndf = df
normalizer = Normalizer()
ndf.iloc[:,0:6] = normalizer.fit_transform(ndf.iloc[:,0:6])
ndf

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2,tipo
0,0.094692,3.921065e-07,0.426150,3.298092e-08,0.899683,2.931638e-08,0
1,0.094652,2.781233e-07,0.426157,1.345758e-07,0.899684,1.076606e-07,0
2,0.094705,4.752663e-08,0.426155,3.485286e-08,0.899679,3.485286e-08,0
3,0.094733,1.202195e-06,0.426058,5.289656e-07,0.899722,4.808778e-07,0
4,0.094692,1.632621e-07,0.426114,1.255863e-07,0.899700,1.130276e-07,0
...,...,...,...,...,...,...,...
11993,0.000200,1.145437e-03,0.000901,1.046589e-01,0.001905,9.945053e-01,5
11994,0.000244,1.470647e-03,0.001103,1.357800e-01,0.002333,9.907345e-01,5
11995,0.000309,1.236239e-03,0.001396,1.111162e-01,0.002952,9.938012e-01,5
11996,0.000314,1.173756e-03,0.001418,1.099357e-01,0.002993,9.939324e-01,5


In [20]:
#drop type column to obtain the x_train data use 6 if size included 3 if not
x_data = df.iloc[:, 0:6]
x_data

,tamano,tiempo,tamano.1,tiempo.1,tamano.2,tiempo.2
0,0.094692,3.921065e-07,0.426150,3.298092e-08,0.899683,2.931638e-08
1,0.094652,2.781233e-07,0.426157,1.345758e-07,0.899684,1.076606e-07
2,0.094705,4.752663e-08,0.426155,3.485286e-08,0.899679,3.485286e-08
3,0.094733,1.202195e-06,0.426058,5.289656e-07,0.899722,4.808778e-07
4,0.094692,1.632621e-07,0.426114,1.255863e-07,0.899700,1.130276e-07
...,...,...,...,...,...,...
11993,0.000200,1.145437e-03,0.000901,1.046589e-01,0.001905,9.945053e-01
11994,0.000244,1.470647e-03,0.001103,1.357800e-01,0.002333,9.907345e-01
11995,0.000309,1.236239e-03,0.001396,1.111162e-01,0.002952,9.938012e-01
11996,0.000314,1.173756e-03,0.001418,1.099357e-01,0.002993,9.939324e-01


In [23]:
#transform type column to categorical vector
y_data = to_categorical(df['tipo'])
y_data

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [24]:
#reshape data to the type the AI model needs
x_data, y_data = np.array(x_data), np.array(y_data)
x_data = np.asarray(x_data).astype('float32')
y_data = np.asarray(y_data).astype('float32')

In [25]:
#Split the data to allow train and test the model
#stratify te hace que haya una proporcion igual de cada categoria en ambos sets a ver klkpasa
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, stratify=y_data)

In [34]:
#PARAMETERS FOR FINE TUNING THE NETWORK
LR = 5e-2                            # learning rate of the gradient descent
LAMBD = 3e-2                         # lambda in L2 regularizaion
DP = 0.0                             # dropout rate
RDP = 0.0                            # recurrent dropout rate
EPOCH = 69                           # number of epochs

initializer = keras.initializers.HeNormal()

#GENERATE MODEL
model = Sequential()

model.add(LSTM(input_shape=(6, 1), units=64,
               activation='relu', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False, 
               kernel_initializer=initializer
              ))
model.add(LSTM(units=64,
               activation='relu', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False, 
               kernel_initializer=initializer
              ))
model.add(LSTM(units=64,
               activation='relu', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(LAMBD), recurrent_regularizer=l2(LAMBD),
               dropout=DP, recurrent_dropout=RDP,
               return_sequences=False, return_state=False,
               stateful=False, unroll=False, 
               kernel_initializer=initializer
              ))
model.add(Dense(units=6, activation='sigmoid'))

model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 6, 64)             16896     
                                                                 
 lstm_13 (LSTM)              (None, 6, 64)             33024     
                                                                 
 lstm_14 (LSTM)              (None, 64)                33024     
                                                                 
 dense_4 (Dense)             (None, 6)                 390       
                                                                 
Total params: 83,334
Trainable params: 83,334
Non-trainable params: 0
_________________________________________________________________


In [76]:
# Define Early Stopping:
early_stop = EarlyStopping(monitor='val_acc', min_delta=0, 
                           patience=69, verbose=1, mode='auto',
                           baseline=0, restore_best_weights=True)
# Define a learning rate decay method:
lr_decay = ReduceLROnPlateau(monitor='loss', 
                             patience=1, verbose=0, 
                             factor=0.5, min_lr=1e-8)
model.fit(x_train, y_train, batch_size=1, epochs=EPOCH, validation_data=(x_validation_data, y_validation_data))

Epoch 1/69
8998/8998 [==============================] - 43s 5ms/step - loss: 1.0921 - accuracy: 0.4734 - val_loss: 18135118.0000 - val_accuracy: 0.1250
Epoch 2/69
8998/8998 [==============================] - 43s 5ms/step - loss: 1.0914 - accuracy: 0.4822 - val_loss: 32274684.0000 - val_accuracy: 0.0625
Epoch 3/69
8998/8998 [==============================] - 42s 5ms/step - loss: 1.0893 - accuracy: 0.4840 - val_loss: 19485490.0000 - val_accuracy: 0.0625
Epoch 4/69
8998/8998 [==============================] - 43s 5ms/step - loss: 1.0922 - accuracy: 0.4702 - val_loss: 60019.5469 - val_accuracy: 0.2500
Epoch 5/69
8998/8998 [==============================] - 42s 5ms/step - loss: 1.0911 - accuracy: 0.4750 - val_loss: 4164393.5000 - val_accuracy: 0.1250
Epoch 6/69
8998/8998 [==============================] - 42s 5ms/step - loss: 1.0919 - accuracy: 0.4751 - val_loss: 53856.3242 - val_accuracy: 0.2500
Epoch 7/69
8998/8998 [==============================] - 42s 5ms/step - loss: 1.0916 - accuracy:

KeyboardInterrupt: 

In [36]:
model.save("modelRNCN.h5")

In [56]:
#debug function to transform order into words
def numToType( t):
  if t==0: return "CTE"
  elif t==1:return "LOG"
  elif t==2:return "N"
  elif t==3:return "NLOGN"
  elif t==4:return "N^2"
  elif t==5:return "N^3"

In [78]:
validation_data_predictions = model.predict_(x_validation_data)
validation_data_predictions

array([[1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.]], dtype=float32)

In [75]:
real_preds = []
for i in range(0,len(validation_data_predictions)):
  p = validation_data_predictions[i]
  if max(p) == p[0]:
    real_preds.append([1,0,0,0,0,0])
  elif max(p) == p[1]:
    real_preds.append([0,1,0,0,0,0])
  elif max(p) == p[2]:
    real_preds.append([0,0,1,0,0,0])
  elif max(p) == p[3]:
    real_preds.append([0,0,0,1,0,0])
  elif max(p) == p[4]:
    real_preds.append([0,0,0,0,1,0])
  elif max(p) == p[5]:
    real_preds.append([0,0,0,0,0,1])

#Calculate error generated in the "real"predictions of test data
err = y_validation_data - real_preds
cont = 0
for i in range(0, len(err)):
  if max(err[i] != min(err[i])):
      cont += 1

print(str(cont) + " / " + str(len(err)) + " err%: " + str(cont / len(err)))

15 / 16 err%: 0.9375
